In [ ]:
import numpy as np
import pandas as pd
import glob

import analysis

# Read all files

In [ ]:
all_files = glob.glob("./results/*.csv")

data = {}
for file in all_files:
    data[file.split("\\")[1]] = np.genfromtxt(file, delimiter=",")

# Optimize for all experiments

In [ ]:
full_positive_matrix, full_negative_matrix = analysis.generate_truthtables()

all_data = []
for i, dat in data.items():

    posdata = np.where(full_positive_matrix, dat, np.nan)
    negdata = np.where(full_negative_matrix, dat, np.nan)

    threshold_width = np.nanmin(posdata) - np.nanmax(negdata)
    threshold_mid = (np.nanmin(posdata) + np.nanmax(negdata))/2
    all_data.append([threshold_width, threshold_mid,  i, dat])
sorted_data = list(sorted(all_data, key=lambda x: x[0], reverse=True))


for i in range(5):
    analysis.plot_hists(sorted_data[i][3], full_positive_matrix, full_negative_matrix, title=sorted_data[i][2]).show()
    analysis.plot_matrix(sorted_data[i][3]).show()

### Performance of parameters

In [ ]:
fig = analysis.reference_matrix(full_positive_matrix, full_negative_matrix)
fig.show()
fig.write_image("./results_analysis/unconstrained_reference_matrix.svg")

In [ ]:
params = [
    [6, 8, 10],
    ["linear", "log", "none"],
    ["0-5", "0-7", "0-9"]
]
parameters = [f"KMR{k}_WGT{w}_TSH{t}" for k in params[0] for w in params[1] for t in params[2]]

for param in parameters:
    idata = data[f"{param}.csv"]

    fig = analysis.plot_hists(idata, full_positive_matrix, full_negative_matrix, title=param)
    # fig.show()
    fig.write_image(f"./results_analysis/unconstrained_{param}_hist.svg")
    fig = analysis.plot_matrix(idata)
    # fig.show()
    fig.write_image(f"./results_analysis/unconstrained_{param}_matrix.svg")

# Constrained case

In [ ]:
to_exclude = set([16, 17, 18, 19, 20, 21, 22, 23, 24, 27, 28, 33, 34, 37, 38])
constrained_positive_matrix, constrained_negative_matrix = analysis.generate_truthtables(to_exclude)

In [ ]:
fig = analysis.reference_matrix(constrained_positive_matrix, constrained_negative_matrix)
fig.show()
fig.write_image("./results_analysis/constrained_reference_matrix.svg")

### Performance of parameters

In [ ]:
params = [8, "linear", "0-7"]
param_string = f"KMR{params[0]}_WGT{params[1]}_TSH{params[2]}"

fig = analysis.plot_hists(data[f"{param_string}.csv"], constrained_positive_matrix, constrained_negative_matrix, title=param_string)
fig.show()
fig.write_image(f"./results_analysis/constrained_{param_string}_hist.svg")
fig = analysis.plot_matrix(data[f"{param_string}.csv"])
fig.show()
fig.write_image(f"./results_analysis/constrained_{param_string}_matrix.svg")